# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** *F*

**Names:**

* *Dessimoz Frank*
* *Micheli Vincent*
* *Lefebvre Hippolyte*

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl
import collections 
from collections import OrderedDict
import operator
courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

## Exercise 4.1: Pre-processing

First let's gather all the courses in a list. We pay attention to not include several times the same ID, we thus look for unique ids.

In [2]:
Ids_courses = list({c['courseId']:c for c in courses})#list of course ids

Let's have a look to that list: 

In [3]:
Ids_courses[0:5]

['MSE-440', 'BIO-695', 'FIN-523', 'MICRO-614', 'ME-231(a)']

In [4]:
len(Ids_courses)

854

We notice a course called "Caution, these contents corresponds to the coursebooks of last year". Let's create two list of courses, one with and one without this "weird" ID:

In [5]:
courses_with =list({c['courseId']:c for c in courses}.values())

In [6]:
courses_without = list({c['courseId']:c for c in courses if not c['courseId']== 'Caution, these contents corresponds to the coursebooks of last year'}.values())

The same way we create a list of unique stopwords:

In [7]:
stop = set(stopwords)#get unique word from stopwords

If we look at stop dictionnary, we notice that there is no ponctuation. Let's create our own dictionnary of them:

In [8]:
import string
punct=[]
for c in string.punctuation:
    punct.append(c)#create the array of punctuation character

In [9]:
punct[0:5]#6 first characters

['!', '"', '#', '$', '%']

Let's update our stop words list with punctuation:

In [10]:
stop.update(punct)#update stop words list with punctuation

Now we want to remove stop words and punctuation and compute word frequency for each document:

In [11]:
from nltk.tokenize import wordpunct_tokenize #ntlk library to tokenize each course description

In [88]:
#We create a dict with course Id as key and words as value
word_dict = {}

for id, course in enumerate(courses_without):
    
    desc = course['description'] #get the dexcription of the current course
    w = []
    for words in wordpunct_tokenize(desc):# tokenize (split) words of the description
        
        word = words.lower() #lower the words to avoid mistakes
        
        if word not in stop:#if word not in stopwords
             
            w.append(word)#We store this word in a list    
         
    #We notice some wrong strings containing two punctuation, we need to split and remove         
    w = [''.join(c for c in s if c not in punct) for s in w]#remove some words containing several punctuations
    w = [s for s in w if s]#remove empty strings caused by the line above
   
    word_dict[course['courseId']]=w#for each courses, we provide the list of words

In [89]:
len(word_dict.keys())

853

In [73]:
list(set(word_dict.keys()))[0:5]

['MGT-641(b)', 'FIN-504', 'COM-303', 'MSE-231', 'COM-308']

Let's have a look to the list of words of the first course called "AR-201(c)":

In [74]:
word_dict['AR-201(c)'][0:9]#9 first

['house',
 'simple',
 'topic',
 'studio',
 'matter',
 'simple',
 'complexity',
 'defining',
 'space']

We still notice some wrong words such as simple numbers. We decide to overlook them, the number of them is low.

We perform stemming to remove suffixes:

In [75]:
from nltk.stem.porter import *# We use porter algo : http://snowball.tartarus.org/algorithms/porter/stemmer.html

In [76]:
stemmer=PorterStemmer()#stemming function
stemmed_words = []#array to collect stemmed words
word_dict_stem={}#dict to store stemmer words for each course

for course, val in word_dict.items():#we loop over each course in the dictionnary of words
    words = word_dict[course]#we get the current course word's list
    
    temp = []
    for w in words:#loop over words
        temp.append(stemmer.stem(w))#we store the stemmed part of the word
        
    stemmed_words+=temp#we fill the list for all stemmed words that will be used later for the matrix
    word_dict_stem[course] = temp#we fill the dict of stemmed words for each course
   

Now we want to compute the count of each word in each course description:

In [77]:
word_dict_count={}#create dictionnary of counts
for course, val in word_dict_stem.items():#loop over each course
    count=collections.Counter(val)#Counter methods returns an array with the count of each word
    word_dict_count[course]=dict(count)#fill the dict of words count per course
  

And eventually the frequency, which is simply those counts divided by the length:

In [78]:
word_dict_freq={}#Not really relevant, TF "frequency" is actually counts
for course, val in word_dict_count.items():
   
    total = sum(val.values(), 0.0)
    new_val = {k: v / total for k, v in val.items()}
    word_dict_freq[course]=new_val
   
    


## Exercise 4.2: Term-document matrix

Let's create the TF matrix that will gather the frequency (which is the counting) of each words in each document:

In [79]:
terms=list(set(stemmed_words))#create a list of unique stemmed words, all stemmed words
classes = list(set(word_dict.keys()))#create a list of unique courses

We dump the list of words for later usage:

In [80]:
with open("terms.pkl", "wb") as list_terms:
    pickle.dump(terms, list_terms)

We can create the matrix:

In [81]:
Matrix_TF = np.zeros((len(terms), len(classes)))#create matrix of term frequency

Check size of marix:

In [62]:
Matrix_TF.shape

(11816, 853)

Now we fill the matrix:

In [82]:
for idc, course in enumerate(word_dict_count):#loop over all courses
    
    for word, freq in word_dict_count[course].items():#loop over each key:value
        
        idw=terms.index(word)#check position of current word in terms array defined above to position in matrix
        Matrix_TF[idw][idc] = freq#fill the matrix
        

The very first word to be processed is "latest":

In [64]:
terms.index('latest')

11481

In [65]:
Matrix_TF[11481][0]

1.0

Now let's compute the inverse document frequency: first we compute counts

In [83]:
word_dict_inv_count={key:0 for key in terms}# we create a dict with our list of stemmed words as keys and null values
for word in terms:#for each words in the stemmed list
   
    for k,v in word_dict_freq.items():#we loop over all doc to find if they contain at least once the word
       
        if word in v:#the key exists in that doc
            word_dict_inv_count[word]+=1#We add one each time a topic appears at least once in a doc
            
    

And apply log to the inverse:

In [84]:
word_dict_inv_freq={}
for k, v in word_dict_inv_count.items():
    word_dict_inv_freq[k]=np.log(len(classes)/v)

In [31]:
len(word_dict_inv_freq)

11816

We can now compute the IDF vector  :

In [85]:
IDF=list(word_dict_inv_freq.values())#IDF vector

And compute the TFIDF matrix which is simply the product of TF matrix and IDF vector:

In [86]:
TF_IDF = Matrix_TF * np.array([IDF]).T

Dump to pickle:

In [87]:
with open("tf_idf.pkl", "wb") as tfidf:
    pickle.dump(TF_IDF, tfidf)

Now let's have a look to the results for our Internet Analytics class: COM 308:

In [34]:
index_IA = Ids_courses.index('COM-308')#find the index of this course

In [35]:
index_IA

43

We want to retrieve all the words for this class:

In [36]:
scores_IA = {}#store words as key and scores as values
for idx, value in enumerate(TF_IDF[:,index_IA]):
    
    scores_IA[terms[idx]] = value

In [37]:
from collections import OrderedDict

In [38]:
Scores_ordered_IA = OrderedDict(sorted(scores_IA.items(),key = operator.itemgetter(1),reverse = True))

In [39]:
list(Scores_ordered_IA.items())[0:14]#15 first topics

[('mine', 18.677272023247372),
 ('onlin', 17.453315047350987),
 ('social', 15.689208174237272),
 ('explor', 15.055449646041554),
 ('world', 14.38779268291894),
 ('hadoop', 12.111224733863468),
 ('real', 11.414257144185477),
 ('servic', 10.839795994687588),
 ('auction', 10.724930372743577),
 ('commerc', 10.724930372743577),
 ('retriev', 9.6056987968727316),
 ('internet', 9.6056987968727316),
 ('network', 9.3681612009097037),
 ('dataset', 8.527705795407357)]

## Exercise 4.3: Document similarity search

Now we are interested in the two follwing subjects : Markov chain and Facebook:

In [40]:
for word in ['markov','chain','facebook']:
    if word in terms:
        print(word + ': True')

markov: True
chain: True
facebook: True


Thus the 3 words belong to our word list.

Now we need to create a vector of Markov chain and Facebook :

In [41]:
MC = np.zeros(len(terms))
FB= np.zeros(len(terms))
MC_FB=np.zeros(len(terms))

#For "markov chain" : two words
MC[terms.index('markov')] = 1/2
MC[terms.index('chain')] = 1/2

#For "facebook"
FB[terms.index('facebook')] = 1

#For markov chain and facebook = 3 words
MC_FB[terms.index('markov')] = 1/3
MC_FB[terms.index('chain')] = 1/3
MC_FB[terms.index('facebook')] = 1/3

Now for this 3 situation we want to find the topics most relevant:

First we define a cosine similarity function (we could also use sklearn method):

In [42]:
def cosine_similarity(di, dj):#compute closeness between a topic and a document
    score= np.dot(di, dj) / (np.linalg.norm(di) * np.linalg.norm(dj))# Frobenius norm byt default
    return score

Then a function to get the 5 top matches by applying the cosine function above:

In [55]:
def top_five(vect):
   
    cosine_score=np.apply_along_axis(cosine_similarity, 0, TF_IDF, vect)#apply function columnwise (each column is a doc)
    top5 = np.argsort(cosine_score)[-5:][::-1]#sort and get five best score
    print(top5)
    temp = []
    for top in top5:
        temp.append((courses_without[top]['courseId'],courses_without[top]['name'], cosine_score[top]))
        #we retrieve the courses in the original course dataframe
    
    print("Five top matches :")
    iterator=1
    for idx,name,score in temp:
        print(str(iterator)+') ', idx,':',name, ':', score) 
        iterator+=1

For Markov chain: 

In [56]:
top_five(MC)

[ 80 398  44 245  99]
Five top matches :
1)  MATH-332 : Applied stochastic processes : 0.552014373903
2)  MGT-484 : Applied probability & stochastic processes : 0.545140925407
3)  MGT-526 : Supply chain management : 0.37517637849
4)  COM-516 : Markov chains and algorithmic applications : 0.375059076082
5)  MGT-602 : Mathematical models in supply chain management : 0.31014411071


For facebook:

In [180]:
top_five(FB)

Five top matches :
1)  EE-727 : Computational Social Media : 0.178554615848
2)  ENV-523 : Hydrogeophysics : 0.0
3)  PHYS-615 : Electronic properties of solids and superconductivity : 0.0
4)  MSE-656 : CCMX Advanced Course - Instrumented Nanoindentation : 0.0
5)  CH-312 : Molecular and cellular biophysic II : 0.0


For both:

In [169]:
top_five(MC_FB)

Five top matches :
1)  MATH-332 : Applied stochastic processes : 0.450717848915
2)  MGT-484 : Applied probability & stochastic processes : 0.445105701718
3)  MGT-526 : Supply chain management : 0.306330230282
4)  COM-516 : Markov chains and algorithmic applications : 0.306234453267
5)  MGT-602 : Mathematical models in supply chain management : 0.253231605989


We notice that for markov chain, finding topics is rather easy while facebook matching only retrieve one match. When we try to get a match for markov chain and facebook at the same time, we get the same results than for only markov chain. At EPFL, since there are a lot of classes dealing with markov chain,these two words appear a lot in course description. Whereas facebook doesn't appear a lot, but social media does:

In [177]:
Soc_media=np.zeros(len(terms))
Soc_media[terms.index('social')] = 1/2
Soc_media[terms.index('media')] = 1/2

In [178]:
top_five(Soc_media)

Five top matches :
1)  EE-727 : Computational Social Media : 0.759554271309
2)  EE-593 : Social media : 0.402063305318
3)  HUM-432(a) : How people learn I : 0.24609192435
4)  COM-308 : Internet analytics : 0.215365138079
5)  EE-552 : Media security : 0.186856249464


So we can say that vector space retrieval models use a lot word direct matching instead of understanding theme or context. 